# Input

In [134]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')
import xgboost
import lightgbm as lgm
import catboost

In [135]:
train = pd.read_csv('train.csv' , parse_dates= ['date_created', 'user_date_created'] )
test = pd.read_csv('test.csv' ,parse_dates= ['date_created', 'user_date_created'] )

# Pre

In [136]:
train.head()

,comment_id,company_id,user_id,product_id,comment,rating,date_created,user_date_created,user_ip,user_agent,is_fake
0,1,2169053,91933,6.0,компания не выполнила заказ без обьяснения пр...,1,2014-10-23 20:26:44,2010-02-03 21:03:22,178.92.70.220,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,False
1,2,709306,91933,2.0,"Впервые встретился такой професиональный, отве...",5,2016-12-10 08:38:27,2010-02-03 21:03:22,94.179.59.162,Mozilla/5.0 (Linux; Android 5.1.1; C6602 Build...,False
2,3,35287,91933,NaN,"Несерйозная, некоректная менеджер інтернет маг...",1,2017-02-11 15:55:14,2010-02-03 21:03:22,37.73.137.188,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False
3,4,1778322,91933,1.0,"Не рекомендую. Не отвечают, не выполняют заказы.",1,2017-06-26 18:00:45,2010-02-03 21:03:22,37.73.149.180,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False
4,5,2821533,91933,NaN,"На письма не отвечают, заказы не выполняют. Не...",1,2017-07-06 16:57:40,2010-02-03 21:03:22,37.73.221.126,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False


In [137]:
startTime_user = train['user_date_created'].min()
y = train['is_fake'].astype(int)

In [195]:
X = pd.DataFrame(index = train.index)
X['company_id'] = train['company_id']
X['product_id'] = train['product_id']
X['rating'] = train['rating']
X = pd.concat([X ,pd.read_csv('train_agent_and_comment.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
X = pd.concat([X ,pd.read_csv('ip_all_train.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
X = pd.concat([X ,pd.read_csv('comment_train.csv')]  , axis =1 ) 
X = pd.concat([X ,pd.read_csv('company_train.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
X = pd.concat([X ,pd.read_csv('user_date_train.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
X = pd.concat([X ,pd.read_csv('time_train.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
X.rename( columns = {'log_coment_agent' : 'log_comment_agent' , 'lens_comments' : 'lens_comment'} , inplace= True )

In [139]:
test_X = pd.DataFrame(index = test.index)
test_X['company_id'] = test['company_id']
test_X['product_id'] = test['product_id']
test_X['rating'] = test['rating']
test_X = pd.concat([test_X ,pd.read_csv('test_agent_and_comment.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
test_X = pd.concat([test_X ,pd.read_csv('ip_all_test.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
test_X = pd.concat([test_X ,pd.read_csv('comment_test.csv')]  , axis =1 ) 
test_X = pd.concat([test_X ,pd.read_csv('company_test.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
test_X = pd.concat([test_X ,pd.read_csv('user_date_test.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
test_X = pd.concat([test_X ,pd.read_csv('time_test.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 

In [140]:
X.head()

,company_id,product_id,rating,log_comment,log_agent,log_coment_agent,agent_encode,ip_false_norm_old,ip_true_values,ip_false_values,...,delta_comm_user_hour,day_comm,month_comm,hour_comm,dayofweek_comm,day_user,month_user,hour_user,dayofweek_user,delta_comm_user_day
0,2169053,6.0,1,0.000664,0.000603,6.094652e-05,12049,0.014124,50211.0,279.0,...,41351.389444,23,10,20,3,3,2,21,2,1722
1,709306,2.0,5,0.001020,0.000498,4.613379e-05,2437,0.006003,22584.0,92.0,...,60035.584722,10,12,8,5,3,2,21,2,2501
2,35287,NaN,1,0.000657,0.000064,2.298313e-06,2103,0.000000,1.0,0.0,...,61554.864444,11,2,15,5,3,2,21,2,2564
3,1778322,1.0,1,0.000301,0.000081,8.855684e-07,2103,0.000000,1.0,0.0,...,64796.956389,26,6,18,0,3,2,21,2,2699
4,2821533,NaN,1,0.000100,0.000078,3.351581e-06,2103,0.000000,1.0,0.0,...,65035.905000,6,7,16,3,3,2,21,2,2709


In [141]:
test_X.head()

,company_id,product_id,rating,log_comment,log_agent,log_comment_agent,agent_encode,ip_false_norm_old,ip_true_values,ip_false_values,...,delta_comm_user_hour,day_comm,month_comm,hour_comm,dayofweek_comm,day_user,month_user,hour_user,dayofweek_user,delta_comm_user_day
0,2214161,1301549.0,5,0.001709,0.000597,0.022145,12049,0.0,1.0,0.0,...,3333.589167,19,3,21,0,1,11,0,2,138
1,2333068,699700.0,5,0.000019,0.000408,0.000002,2437,0.0,1.0,0.0,...,949.741667,10,12,13,6,1,11,0,2,39
2,752692,NaN,5,0.002556,0.000069,0.000027,2103,0.1,29.0,5.0,...,0.081667,1,11,0,2,1,11,0,2,0
3,334751,1279247.0,5,0.000755,0.000069,0.000055,2103,-1.0,1.0,0.0,...,479.621389,20,11,23,0,1,11,0,2,19
4,2263650,1225545.0,5,0.009095,0.000069,0.000030,2103,-1.0,1.0,0.0,...,479.618611,20,11,23,0,1,11,0,2,19


In [142]:
corM =  pd.concat([X , y ], axis =1).corr()

In [143]:
corM['is_fake']

company_id                   0.020309
product_id                   0.103157
rating                       0.026522
log_comment                  0.613987
log_agent                    0.179849
log_coment_agent             0.547152
agent_encode                 0.022523
ip_false_norm_old            0.533109
ip_true_values               0.010881
ip_false_values              0.145667
ip_false_norm                0.555237
ip_total_values              0.011917
ip_true_values_new           0.037793
ip_false_values_new          0.259517
ip_false_norm_new            0.622617
ip_total_values_new          0.042055
lens_comments               -0.023367
nb_words                    -0.023718
nb_unique_words             -0.024221
freq_words                   0.013931
company_true                -0.019621
company_false                0.735671
company_overall              0.010649
company_false_normalized     0.762048
company_rate_total          -0.020909
company_average_rate        -0.255436
company_rati

# Val

In [144]:
from sklearn.model_selection import StratifiedKFold , cross_val_score , GridSearchCV
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42 )

# Models

In [145]:
import lightgbm as lgb
lgbm_train = lgb.Dataset(X, label = y , free_raw_data = False )
n_rounds = 10000

In [147]:
modelLGB = lgb.LGBMClassifier(
    objective = 'binary' ,
    learning_rate = 0.05,
    num_threads=  -1,
    #'metric': 'loss',
    
    
    colsample_bytree =0.9,
    subsample = 0.8,
    subsample_freq =1,
    n_estimators = 60
)
                      

In [148]:
modelLGB.fit(X,y)
submission =  pd.DataFrame( modelLGB.predict_proba(test_X)[:,1] , index = test['comment_id'] , columns = ["is_fake"] )
submission.to_csv('submission_Boosting.csv')

----

In [179]:
import xgboost as xgb
from xgboost import XGBClassifier
xgb_train = xgb.DMatrix(X, y, feature_names=X.columns)
num_rounds = 200

In [180]:
from xgboost import XGBClassifier
modelXg = XGBClassifier(
    eta = 0.1,
    eval_metric = 'logloss',
    
    # regularization parameters
    max_leaves = 32,
    subsample = 0.7,
    colsample_bytree = 0.7,
    
    #lightgbm approach
    tree_method = 'hist',
    grow_policy = 'lossguide',
    learning_rate =0.05,
    n_estimators=60)

In [198]:
modelXg.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, eta=0.1, eval_metric='logloss', gamma=0,
       grow_policy='lossguide', learning_rate=0.05, max_delta_step=0,
       max_depth=3, max_leaves=32, min_child_weight=1, missing=None,
       n_estimators=60, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.7, tree_method='hist')

In [212]:
submissionxg =  pd.DataFrame(  modelXg.predict_proba(test_X  )[:,1], index = test['comment_id'] , columns = ["is_fake"] )
submissionxg.to_csv('submissionxg_Xg.csv')

---

In [ ]:
import catboost as ctb
ctb_model = ctb.CatBoostClassifier(iterations = 40, learning_rate=0.05, depth=4, eval_metric='Logloss', 
                                       random_seed=42, rsm=0.8, l2_leaf_reg=2, od_wait=50, od_type='Iter' , verbouse = 0)

In [126]:
ctb_model.fit(X, y)
submissionctb =  pd.DataFrame(  ctb_model.predict_proba(test_X)[:,1], index = test['comment_id'] , columns = ["is_fake"] )
submissionctb.to_csv('submissionctb_Cat.csv')

In [209]:
from sklearn.ensemble import RandomForestClassifier , ExtraTreesClassifier
from sklearn.model_selection import cross_val_predict

In [224]:
models = {
          #'blg' : ,
          'rf' :RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1, verbose = 2 ,max_depth=17,min_samples_leaf=17 ) ,
          'et' :ExtraTreesClassifier(n_estimators=50, random_state=42, n_jobs=-1, verbose = 2 ,max_depth=17,min_samples_leaf=17 , max_features = 0.8) , 
          #'xg' : modelXg ,
          #'light' : modelLGB,
          #'cat' : ctb_model
          #'keras' : ,
         }

In [ ]:
prediction = pd.DataFrame()
for modelName in models:
    print(modelName)
    prediction[modelName] =  cross_val_predict(models[modelName] , X.fillna(-1) , y , cv = skf, verbose =2, method='predict_proba')


In [259]:
prediction['lgb'] = pd.read_csv('lgb_pred.csv')['is_fake']
prediction['xg'] = pd.read_csv('xg_pred.csv')['is_fake']
prediction['cat'] = pd.read_csv('cat_pred.csv')['is_fake']
prediction['rf'] = pd.read_csv('submisson_RF.csv')['is_fake']
prediction['lgb_s'] = pd.read_csv('lgb_s.csv')['is_fake']
prediction['vova'] = pd.read_csv('submisson_35.csv')['is_fake']

In [255]:
abc = ((prediction['lgb'] + prediction['xg'] + prediction['cat'] + prediction['rf'] + prediction['lgb_s'])/5)
abc1 = (( prediction['rf'] + prediction['lgb_s'])/2)
abc2 = (np.sqrt( prediction['rf'] * prediction['lgb_s']))


In [262]:
abc3 =prediction['vova']*0.5 + prediction['rf']*0.2 + prediction['lgb_s']*0.3

In [263]:
submissionctb =  pd.DataFrame(  list(abc3) , index = test['comment_id'] , columns = ["is_fake"] )
submissionctb.to_csv('submissionctb_F.csv')

In [248]:
submissionctb

,is_fake
comment_id,
1655822,0.004436
1655823,0.004261
1655824,0.012577
1655825,0.004260
1655826,0.005644
1655827,0.004265
1655828,0.004265
1655829,0.005389
1655830,0.014188
